<a href="https://colab.research.google.com/github/yhytoto12/SNA/blob/master/KarateClubVisualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-geometric

In [155]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch_geometric.transforms as T
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import Flickr
from sklearn.manifold import TSNE

from torch_geometric.datasets import KarateClub
dataset = KarateClub()
#dataset = Planetoid('./data', 'PubMed')
#dataset = Flickr('./data')
data = dataset[0]

device = 'cuda' if torch.cuda.is_available() else 'cpu'

colors = [
    '#ff0000', '#0000ff'
]

### Make Training Set

In [ ]:
random.seed(1)
alpha = 0.3
mask = random.sample(range(34), int(alpha * 34))
train_mask = np.full(34, False, dtype=bool)
for i in mask:
    train_mask[i] = True

print(train_mask)

# Node2Vec

In [ ]:
from torch_geometric.nn import Node2Vec


model = Node2Vec(data.edge_index, embedding_dim=10, walk_length=10,
                 context_size=5, walks_per_node=10, num_negative_samples=1,
                 q=1,
                 sparse=True).to(device)

loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.SparseAdam(model.parameters(), lr=0.01)


def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


@torch.no_grad()
def test():
    model.eval()
    z = model()
    acc = model.test(z[train_mask], data.y[train_mask], z, data.y, max_iter=100)
    return acc


for epoch in range(1, 201):
    loss = train()
    acc = test()
    if epoch % 20 == 0:
        print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Acc: {acc:.4f}')




### Plot Embedding Results

In [ ]:
@torch.no_grad()
def plot_points(colors):
    model.eval()
    z = model(torch.arange(data.num_nodes, device=device))
    z = TSNE(n_components=2, perplexity=2.5).fit_transform(z.cpu().numpy())
    y = data.y.cpu().numpy()

    plt.figure(figsize=(8, 8))
    for i in range(dataset.num_classes):
        plt.scatter(z[y == i, 0], z[y == i, 1], s=20, color=colors[i])
    plt.axis('off')
    plt.show()


plot_points(colors)

In [ ]:
from torch_geometric.utils import to_networkx
import networkx as nx
g = to_networkx(data, to_undirected=True)
colormap = ['#ff0000' if data.y[i] == 0 else '#0000ff' for i in range(34) ]
plt.figure(figsize=(8,8))
plt.axis('off')
nx.draw_networkx(g,
                 arrows=False,
                 with_labels=False,
                 node_size=100,
                 node_shape='o',
                 node_color=colormap)

# Louvain

In [134]:
import matplotlib.pyplot as plt
import random
import matplotlib.cm as cm
import networkx as nx
import numpy as np
import math
import time
from numpy import linalg as la
from sklearn.metrics import accuracy_score
import community as community_louvain
def Louvain(graph):
    partition = community_louvain.best_partition(graph)
    n_coms = len(set(partition.values()))
    coms = [set() for _ in range(n_coms)]
    for k in partition.keys():
        coms[partition[k]].add(k)
    return coms

In [152]:
from torch_geometric.utils import to_networkx

g = to_networkx(data, to_undirected=True)

num_nodes = data.num_nodes
comms = Louvain(g)
pred = np.zeros(num_nodes)
num_comms = len(comms)

In [153]:
i = 0
for c in comms:
    for j in c:
        pred[j] = i
    i += 1
new_pred = np.copy(pred)
for ci in range(num_comms):
    try:
        label = np.bincount(data.y[np.logical_and(pred == ci, data.train_mask)]).argmax()
    except:
        label = -10
    new_pred[pred == ci] = label

acc = accuracy_score(data.y[data.test_mask].numpy(), new_pred[data.test_mask])
print('Acc : {:.3f}'.format(acc))


Acc : 0.442
